In [104]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision
import torchvision.transforms as transforms
from PIL import Image
import os
import matplotlib.pyplot as plt
import matplotlib.patches as patches

In [241]:
class ImageSet(Dataset):
    def __init__(self, image_paths, image_labels, transforms, root_adj = ''):
        super().__init__()
        self.images_paths = image_paths
        self.transforms = transforms
        self.image_labels = image_labels
        self.root_adj = root_adj # adjust path based on your folder structure

    def __len__(self):
        return len(self.images_paths)
    
    def __getitem__(self, idx):
        image = Image.open(self.root_adj + self.images_paths[idx])
        label = self.image_labels[idx]
        image = self.transforms(image)
        bbox = torch.zeros(2)
        if label == 1:
            bbox = bbox + image.shape[1]
        return image, label, bbox


In [242]:
train_mura = pd.read_csv('../MURA-v1.1/train_image_paths.csv', header=None, names=['paths'])
test_mura = pd.read_csv('../MURA-v1.1/valid_image_paths.csv',header=None, names=['paths'])
train_mura_labels = pd.read_csv('../MURA-v1.1/train_labeled_studies.csv',header=None, names=['labels'])
test_mura_labels = pd.read_csv('../MURA-v1.1/valid_labeled_studies.csv',header=None, names=['labels'])
train_mura['indices'] = train_mura.paths.str[:-10]
test_mura['indices'] = test_mura.paths.str[:-10]
# full MURA with all parts
# can select parts by filtering indices
train_data = pd.merge(train_mura, train_mura_labels, left_on='indices', right_index=True)
test_data = pd.merge(test_mura, test_mura_labels, left_on='indices', right_index=True)

In [243]:
train_mura.paths[0]

'MURA-v1.1/train/XR_SHOULDER/patient00001/study1_positive/image1.png'

In [244]:
resize = 256
trans = transforms.Compose([transforms.Resize((resize, resize)),
                            transforms.Lambda(lambda img: img.convert('L')),
                            transforms.ToTensor()])
mura_dataset = ImageSet(train_data.paths, train_data.labels, trans, root_adj='../')
mura_testset = ImageSet(test_data.paths, test_data.labels, trans)

In [248]:
batch_size = 64
mura_trainloader = DataLoader(mura_dataset, batch_size=batch_size, shuffle=True)
mura_testloader = DataLoader(mura_testset, batch_size=batch_size, shuffle=True)